<a href="https://colab.research.google.com/github/KoganTheDev/cloud-computing-project/blob/main/presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:

!pip install firebase
!pip install numpy
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import time
import matplotlib.pyplot as plt
import io
from IPython.display import display as ipy_display
from typing import Dict, List, Callable,Union,Any
from firebase import firebase
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # Python 3.9+
import dateutil.parser         # Robust timestamp parsing
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo
from nltk.stem import PorterStemmer
local_tz = ZoneInfo("Asia/Jerusalem")
# --- Constants ---
BLUE_COLOR = '#3498db'
DARKER_BLUE_COLOR = '#2c3e50'
ORANGE_COLOR = '#FF851B'
LIGHT_BLUE_COLOR = '#f0f8ff'
LIGHTER_BLUE_COLOR = '#f7f9fc'
WHITE_COLOR = 'rgba(255,255,255,0.7)'
LIGHT_GREY_BORDER = '#ddd'
BOX_SHADOW = '0px 4px 8px rgba(0,0,0,0.2)'
BORDER_RADIUS = '12px'
PADDING = '30px'
MARGIN = '20px'

In [149]:

# --- Data Types ---
SensorData = Dict[str, Dict[str, float or str]]
EngineersPerformance = Dict[str, Dict[str, int]]
HistoricalSensorData = Dict[str, Dict[str, List[float] or List[str]]]

stemmer = PorterStemmer()

# --- Firebase Setup ---
FBconn = firebase.FirebaseApplication(
    'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
)
local_tz = ZoneInfo("Asia/Jerusalem")
DATABASE_PATH = 'Data'
INDOOR_SENSOR_PATH = f'{DATABASE_PATH}/indoor'
OUTDOOR_SENSOR_PATH = f'{DATABASE_PATH}/outdoor'

engineers_performance: EngineersPerformance = {
    "Alice": {"Points": 1550, "Improvements Completed": 32},
    "Bob": {"Points": 1200, "Improvements Completed": 25},
    "Charlie": {"Points": 1800, "Improvements Completed": 40},
    "David": {"Points": 950, "Improvements Completed": 18},
}

# Enhanced Daily Tasks Structure - converted from your original data
daily_tasks = [
    {
        "id": 1,
        "description": "Review production logs for anomalies",
        "points": 155,
        "status": "pending",  # pending, completed
        "completed_by": None,
        "completion_date": None
    },
    {
        "id": 2,
        "description": "Check calibration of robotic arms",
        "points": 120,
        "status": "pending",
        "completed_by": None,
        "completion_date": None
    },
    {
        "id": 3,
        "description": "Optimize energy consumption routines",
        "points": 150,
        "status": "pending",
        "completed_by": None,
        "completion_date": None
    },
    {
        "id": 4,
        "description": "Inspect conveyor belt system",
        "points": 120,
        "status": "pending",
        "completed_by": None,
        "completion_date": None
    }
]

In [150]:
# --- Helper Functions (Refactored for Clarity) ---
def create_html_widget(value: str, **layout_kwargs) -> widgets.HTML:
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HTML(value=value, layout=layout)

def create_button(description: str, button_style: str = 'primary', **layout_kwargs) -> widgets.Button:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '200px'),
        height=layout_kwargs.get('height', '60px'),
        border_radius=layout_kwargs.get('border_radius', BORDER_RADIUS),
        font_size=layout_kwargs.get('font_size', '16px'),
        margin=layout_kwargs.get('margin', '10px'),
        box_shadow=layout_kwargs.get('box_shadow', None)
    )
    return widgets.Button(description=description, button_style=button_style, layout=layout)

def create_text_input(description: str, **layout_kwargs) -> widgets.Text:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '600px'),
        height=layout_kwargs.get('height', '40px'),
        border_radius=layout_kwargs.get('border_radius', BORDER_RADIUS),
        padding=layout_kwargs.get('padding', '10px'),
        font_size=layout_kwargs.get('font_size', '16px')
    )
    return widgets.Text(description=description, layout=layout)

def create_output(**layout_kwargs) -> widgets.Output:
    layout = widgets.Layout(**layout_kwargs)
    return widgets.Output(layout=layout)

def create_dropdown(options: list, description: str, **layout_kwargs) -> widgets.Dropdown:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '300px'),
        margin=layout_kwargs.get('margin', '10px 0')
    )
    return widgets.Dropdown(options=options, description=description, layout=layout)

def create_vbox(children: list, **layout_kwargs) -> widgets.VBox:
    layout = widgets.Layout(
        padding=layout_kwargs.get('padding', PADDING),
        border=layout_kwargs.get('border', f'2px solid {BLUE_COLOR}'),
        border_radius=layout_kwargs.get('border_radius', '15px'),
        margin=layout_kwargs.get('margin', MARGIN),
        background_color=layout_kwargs.get('background_color', LIGHT_BLUE_COLOR)
    )
    return widgets.VBox(children=children, layout=layout)

def create_hbox(children: list, **layout_kwargs) -> widgets.HBox:
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HBox(children=children, layout=layout)

# --- Use HtmlWidgetFactory for all HTML widgets ---
# (No change needed for create_html_widget, already mapped)

# --- Clean up perform_search for clarity ---
def perform_search(query: str):
    stemmed_query = stemmer.stem(query.lower())
    index_data = FBconn.get('/Data/Index', None)
    results = []
    if index_data and stemmed_query in index_data:
        entry = index_data[stemmed_query]
        results.append(f"<h3>Search Result for: <i>{query}</i></h3>")
        results.append(f"<b>Term:</b> {entry['term']}")
        results.append(f"<b>Count:</b> {entry['count']} occurrence(s)")
        results.append("<b>Appears in URLs:</b><ul>")
        # DocIDs is a list of dicts
        results.extend([
            f"<li><a href='{item['url']}' target='_blank'>{item['url']}</a> <span style='color:black;'>({item['count']} times)</span></li>"
            for item in entry.get('DocIDs', [])
        ])
        results.append("</ul>")
    else:
        results.append(f"<b>No matching word</b> found in the top 100 index for: <i>{query}</i>")
    return results

In [151]:
def generate_sensor_statistics(selected_sensor: str, sensor_data: SensorData, selected_graph_attribute: str = None) -> widgets.HTML:
    """
    Generates an HTML widget displaying the latest sensor data for a selected attribute
    along with basic statistical calculations from historical data for that attribute,
    considering only the last 5 minutes of data.
    """
    # Return empty/placeholder message if no sensor or attribute is selected
    if not selected_sensor or selected_sensor == "-- Select Sensor --":
        return create_html_widget("<div style='text-align: center; color: #666; font-style: italic;'>Please select a sensor to view statistics</div>")

    if not selected_graph_attribute or selected_graph_attribute == "-- Select Attribute --":
        return create_html_widget("<div style='text-align: center; color: #666; font-style: italic;'>Please select an attribute to view statistics</div>")

    latest_data = sensor_data.get(selected_sensor, {})
    if not latest_data:
        return create_html_widget(f"No data available for **{selected_sensor}**.")

    units_map = {
        "Indoor Sensor": {"Humidity": "%", "Temperature": "°C", "Pressure": "Bar", "Distance": "Cm"},
        "Outdoor Sensor": {"Humidity": "%", "Temperature": "°C", "DLight": "Lux"}
    }
    unit = units_map.get(selected_sensor, {}).get(selected_graph_attribute, "")

    stats_html = ""
    # Display the latest value for the selected attribute
    if selected_graph_attribute:
        found_key_latest = next((k for k in latest_data if k.lower() == selected_graph_attribute.lower()), None)
        if found_key_latest:
            stats_html += f"<b>Latest {selected_graph_attribute}:</b> {latest_data[found_key_latest]} {unit}<br>"
        else:
            stats_html += f"<b>No latest data available for '{selected_graph_attribute}'.</b><br>"

    # Always try to display the latest Time if available
    if "Time" in latest_data:
        stats_html += f"<b>Latest Time:</b> {latest_data['Time']}</b><br>"

    # Add statistical calculations from historical data (last 5 minutes)
    if selected_graph_attribute:
        sensor_path = INDOOR_SENSOR_PATH if selected_sensor == "Indoor Sensor" else OUTDOOR_SENSOR_PATH
        # Fetch historical data which is now filtered to the last 5 minutes
        historical_data_filtered = fetch_historical_sensor_data(sensor_path)

        # Extract numerical data for the selected attribute, filtering out None values
        attribute_data = [
            val for val in historical_data_filtered.get(selected_graph_attribute, []) if isinstance(val, (int, float))
        ]

        if attribute_data:
            stats_html += "<br><b>Statistics (Last 5 Minutes):</b><br>" # Updated label
            stats_html += f"<b>Mean:</b> {np.mean(attribute_data):.2f} {unit}<br>"
            stats_html += f"<b>Median:</b> {np.median(attribute_data):.2f} {unit}<br>"
            stats_html += f"<b>Min:</b> {np.min(attribute_data):.2f} {unit}<br>"
            stats_html += f"<b>Max:</b> {np.max(attribute_data):.2f} {unit}<br>"
            stats_html += f"<b>Standard Deviation:</b> {np.std(attribute_data):.2f} {unit}<br>"
        else:
            stats_html += f"<br><b>No sufficient numerical data in the last 5 minutes for '{selected_graph_attribute}' to calculate statistics.</b><br>"

    return create_html_widget(f"<div style='display: flex; justify-content: space-between;'><div style='flex: 1; margin-right: 20px;'>{stats_html}</div></div>")


def fetch_latest_sensor_data() -> dict:
    """Fetch the latest value for each attribute from indoor and outdoor sensor data."""

    def get_latest_all_fields(data):
        if not data:
            return {}

        all_possible_keys = set()
        for record in data.values():
            all_possible_keys.update(record.keys())

        latest_values = {}
        sorted_items = sorted(data.items(), key=lambda x: int(x[0]), reverse=True)

        for _, record in sorted_items:
            for key in all_possible_keys:
                if key not in latest_values and key in record:
                    latest_values[key] = record[key]
            if len(latest_values) == len(all_possible_keys):
                break

        return latest_values

    indoor_data = FBconn.get(INDOOR_SENSOR_PATH, None)
    outdoor_data = FBconn.get(OUTDOOR_SENSOR_PATH, None)

    indoor_latest = get_latest_all_fields(indoor_data)
    outdoor_latest = get_latest_all_fields(outdoor_data)

    return {
        "Indoor Sensor": indoor_latest,
        "Outdoor Sensor": outdoor_latest,
    }

def fetch_historical_sensor_data(sensor_path: str) -> HistoricalSensorData:
    """
    Fetches historical sensor data for a given sensor path,
    filtered to include only entries from the last 5 minutes relative to the latest timestamp.
    The data is sorted by timestamp and includes all available attributes.
    """
    data = FBconn.get(sensor_path, None)
    israel_tz = ZoneInfo("Asia/Jerusalem")

    historical_data: HistoricalSensorData = {"Timestamp": [], "Humidity": [], "Temperature": []}
    if sensor_path == INDOOR_SENSOR_PATH:
        historical_data.update({"Pressure": [], "Distance": []})
    elif sensor_path == OUTDOOR_SENSOR_PATH:
        historical_data.update({"DLight": []})

    if not data:
        return historical_data

    # Sort data by timestamp to ensure correct plotting order and to find the latest
    sorted_entries = sorted(
        data.values(),
        key=lambda x: dateutil.parser.parse(f"{x.get('Date', '')} {x.get('Time', '')}") if x.get('Date') and x.get('Time') else datetime.min
    )

    # Determine the latest valid timestamp and the 5-minute threshold
    latest_timestamp = None
    for entry in reversed(sorted_entries): # Iterate in reverse to find the latest valid timestamp
        date_str = entry.get("Date", "")
        time_str = entry.get("Time", "")
        full_timestamp_str = f"{date_str} {time_str}" if date_str and time_str else ""
        try:
            entry_time = dateutil.parser.parse(full_timestamp_str)
            if entry_time.tzinfo is None:
                entry_time = entry_time.replace(tzinfo=ZoneInfo("UTC")).astimezone(israel_tz)
            else:
                entry_time = entry_time.astimezone(israel_tz)
            latest_timestamp = entry_time
            break # Found the latest valid timestamp, exit loop
        except Exception:
            continue # Skip invalid time formats

    if not latest_timestamp:
        return historical_data # No valid timestamps found

    # Calculate the time threshold (5 minutes before the latest timestamp)
    five_minutes_ago = latest_timestamp - timedelta(minutes=5)

    # Filter entries within the last 5 minutes
    for entry in sorted_entries:
        date_str = entry.get("Date", "")
        time_str = entry.get("Time", "")
        full_timestamp_str = f"{date_str} {time_str}" if date_str and time_str else ""
        try:
            entry_time = dateutil.parser.parse(full_timestamp_str)
            if entry_time.tzinfo is None:
                entry_time = entry_time.replace(tzinfo=ZoneInfo("UTC")).astimezone(israel_tz)
            else:
                entry_time = entry_time.astimezone(israel_tz)

            # Only append if the entry is within the last 5 minutes
            if entry_time >= five_minutes_ago and entry_time <= latest_timestamp:
                historical_data["Timestamp"].append(entry_time)
                historical_data["Humidity"].append(entry.get("Humidity"))
                historical_data["Temperature"].append(entry.get("Temperature"))
                if sensor_path == INDOOR_SENSOR_PATH:
                    historical_data["Pressure"].append(entry.get("Pressure"))
                    historical_data["Distance"].append(entry.get("Distance"))
                elif sensor_path == OUTDOOR_SENSOR_PATH:
                    historical_data["DLight"].append(entry.get("DLight"))
        except Exception:
            continue # Skip entries with invalid time formats

    return historical_data


def create_historical_plot(sensor_name: str, selected_attribute: str) -> widgets.Output:
    """Creates and returns an output widget with a single historical plot for the selected attribute.
    Creates a fluid connected line even when data points are missing."""

    # Handle case where no sensor or attribute is selected
    if not sensor_name or sensor_name == "-- Select Sensor --":
        no_selection_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with no_selection_output:
            display(widgets.HTML("<div style='text-align: center; color: #666; font-style: italic; padding: 20px;'>Please select a sensor to view the plot</div>"))
        return no_selection_output

    if not selected_attribute or selected_attribute == "-- Select Attribute --":
        no_selection_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with no_selection_output:
            display(widgets.HTML("<div style='text-align: center; color: #666; font-style: italic; padding: 20px;'>Please select an attribute to view the plot</div>"))
        return no_selection_output

    # Centralized configuration for sensors, units, and attribute-specific colors
    sensor_configurations = {
        "Indoor Sensor": {
            "path": INDOOR_SENSOR_PATH,
            "units": {
                "Humidity": "%",
                "Temperature": "°C",
                "Pressure": "Bar",
                "Distance": "Cm"
            },
            "colors": { # Define specific colors for each attribute of the Indoor Sensor
                "Humidity": 'tab:blue',
                "Temperature": 'tab:red',
                "Pressure": 'tab:green',
                "Distance": 'tab:purple'
            }
        },
        "Outdoor Sensor": {
            "path": OUTDOOR_SENSOR_PATH,
            "units": {
                "Humidity": "%",
                "Temperature": "°C",
                "DLight": "Lux"
            },
            "colors": { # Define specific colors for each attribute of the Outdoor Sensor
                "Humidity": 'tab:orange',
                "Temperature": 'tab:cyan',
                "DLight": 'tab:brown'
            }
        }
    }

    # Get the configuration for the selected sensor
    config = sensor_configurations.get(sensor_name)

    # Handle invalid sensor name
    if not config:
        error_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with error_output:
            display(widgets.HTML(f"<b>❌ Invalid sensor name:</b> {sensor_name}"))
        return error_output

    # Fetch historical data using the path from the configuration (now filtered to last 5 minutes)
    historical_data = fetch_historical_sensor_data(config["path"])
    units = config["units"]
    attribute_colors = config["colors"] # Get the specific colors for this sensor

    # Determine the color for the selected attribute
    color = attribute_colors.get(selected_attribute, 'gray') # Default to gray if no specific color is found

    # Handle case where no historical data is available
    if not historical_data or not historical_data["Timestamp"]:
        no_data_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with no_data_output:
            display(widgets.HTML(f"<b>📭 No historical data available for {sensor_name} in the last 5 minutes.</b>"))
        return no_data_output

    out = widgets.Output()
    with out:
        plt.figure(figsize=(10, 6))

        times = historical_data["Timestamp"]
        y_data = historical_data.get(selected_attribute, [])
        unit = units.get(selected_attribute, "")

        if not y_data:
            # Display message if the attribute exists but has no data within the filtered period
            plt.text(0.5, 0.5, f"No data available for '{selected_attribute}' in the last 5 minutes.", ha='center', va='center', fontsize=12)
            plt.title(f"{selected_attribute} over Time ({sensor_name}) - Last 5 Minutes", fontweight='bold')
        else:
            # Filter out None values to create a fluid connected line
            valid_points = [(t, y) for t, y in zip(times, y_data) if y is not None]

            if valid_points:
                valid_times, valid_y_data = zip(*valid_points)

                # Plot main line
                plt.plot(valid_times, valid_y_data, marker='o', color=color,
                                     linewidth=2, markersize=5, label=selected_attribute)

                # Shadow line (no label) for visual thickness
                plt.plot(valid_times, valid_y_data, color=color, alpha=0.3, linewidth=4)

                # Axis formatting and labels
                plt.title(f"{selected_attribute} over Time ({sensor_name}) - Last 5 Minutes", fontweight='bold') # Updated title
                plt.xlabel("Time", fontweight='bold')
                plt.ylabel(f"{selected_attribute} ({unit})", fontweight='bold')
                plt.grid(True, linestyle='--', alpha=0.4)

                # Format x-axis for datetime objects
                if valid_times and isinstance(valid_times[0], datetime): # Check if times are datetime objects
                    ax = plt.gca()
                    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

                # Add y-axis padding for better visualization
                y_min, y_max = min(valid_y_data), max(valid_y_data)
                if y_min != y_max: # Only apply padding if there's a range
                    y_range = y_max - y_min
                    plt.ylim(y_min - 0.15 * y_range, y_max + 0.15 * y_range)
                else: # Handle case where all y_data points are the same
                    plt.ylim(y_min - 1, y_max + 1) # Small fixed padding

                plt.legend()
            else:
                # This case handles when y_data contains only None values within the filtered period
                plt.text(0.5, 0.5, f"No valid data points for '{selected_attribute}' in the last 5 minutes.",
                                     ha='center', va='center', fontsize=12)
                plt.title(f"{selected_attribute} over Time ({sensor_name}) - Last 5 Minutes", fontweight='bold')

        plt.tight_layout() # Adjust plot to prevent labels from overlapping
        plt.show()
    return out

def update_sensor_data_and_plot(change) -> None:
    """Updates the statistics display and plots based on the selected sensor and attribute."""
    selected_sensor = sensor_selector.value # Get the current sensor selected
    selected_attribute = graph_selector.value # Get the current attribute selected

    latest_data = fetch_latest_sensor_data()
    # Update the display for selected sensor and graph
    # Pass selected_attribute to generate_sensor_statistics
    stats_display.value = generate_sensor_statistics(selected_sensor, latest_data, selected_attribute).value

    # Clear the output widget and display the new figure
    stats_plots.clear_output(wait=True)
    with stats_plots:
        out = create_historical_plot(selected_sensor, selected_attribute)
        display(out)

In [152]:
# --- Main Menu Screen Widgets ---
main_menu_title = create_html_widget(
    "<h1 style='color:#3498db; text-align:center; margin-bottom:30px;'>Robotics Lab Control Center</h1>",
    margin='0 0 20px 0'
)

# Create large, attractive menu buttons
manager_btn = create_button(
    "System Manager",
    button_style='info',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

statistics_btn = create_button(
    "Sensor Statistics",
    button_style='success',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

search_btn = create_button(
    "Search Query",
    button_style='warning',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

optimization_btn = create_button(
    "Optimization Race",
    button_style='danger',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

# Fetch the latest sensor data right before creating the dashboard_summary
# This ensures the status is up-to-date when the dashboard_summary is rendered.
latest_sensor_info = fetch_latest_sensor_data()
indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

# --- FIX STARTS HERE ---
# Calculate pending_tasks_count BEFORE creating the dashboard_summary HTML string
pending_tasks_count = sum(1 for task in daily_tasks if task.get('status') == 'pending')
# --- FIX ENDS HERE ---
leader_name, leader_data = max(engineers_performance.items(), key=lambda item: item[1].get("Points", 0))
leader_score = leader_data["Points"]

# Create dashboard summary for main menu
dashboard_summary = create_html_widget(
    f"""
    <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
        <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
        <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                </h4>
                <p style='margin-bottom:5px; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                </p>
                <p style='margin-bottom:0; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                </p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{pending_tasks_count}</span> tasks pending</p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>{leader_name}</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>{leader_score}</span> points</p>

            </div>
        </div>
    </div>
    """,
    margin='20px 0'
)

# Arrange menu buttons in a grid layout
top_row = create_hbox([manager_btn, statistics_btn], justify_content='center')
bottom_row = create_hbox([search_btn, optimization_btn], justify_content='center')
menu_buttons = create_vbox(
    [top_row, bottom_row],
    border='none',
    background_color='transparent',
    padding='10px'
)

# Welcome message with date/time
current_time = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
welcome_msg = create_html_widget(
    f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time}</p></div>"
)

# Main menu container
main_menu_container = create_vbox(
    [
        main_menu_title,
        welcome_msg,
        dashboard_summary,
        menu_buttons
    ],
    background_color=LIGHTER_BLUE_COLOR,
    border=f'2px solid {DARKER_BLUE_COLOR}',
    box_shadow=BOX_SHADOW,
    width='800px',
    margin='0 auto'
)

In [153]:
# --- System Manager Screen Widgets ---
manager_title = create_html_widget("<h2 style='color:#3498db;'>Robotics Lab System Manager Dashboard</h2>")
production_status_title = create_html_widget("<h3>Sensor Status</h3>")
sensor_status_display = create_html_widget("") # To display indoor/outdoor status
tasks_title = create_html_widget("<h3>Daily Optimization Tasks</h3>")

# Enhanced task list - show description, status, and completion details
from datetime import datetime

def format_optimization_task_list(tasks):
    """Format tasks using the same enhanced styling as the manager screen"""
    from datetime import datetime

    formatted_tasks = []
    for task in tasks:
        if isinstance(task, dict):
            # Extract task details
            task_id = task.get('id', 'N/A')
            description = task.get('description', 'No description')
            status = task.get('status', 'unknown')
            completed_by = task.get('completed_by', None)
            completion_date = task.get('completion_date', None)
            points = task.get('points', 0)

            # Format completion date - if already in DD/MM/YYYY HH:MM format, use as is
            formatted_date = None
            if completion_date:
                try:
                    # If it's already in DD/MM/YYYY HH:MM format, use directly
                    if isinstance(completion_date, str):
                        # Check if it's already in the correct format
                        if '/' in completion_date and len(completion_date.split('/')) == 3:
                            formatted_date = completion_date
                        else:
                            # Try to parse other formats and convert to DD/MM/YYYY HH:MM
                            for fmt in ['%Y-%m-%d %H:%M', '%Y-%m-%d %H:%M:%S', '%Y-%m-%d', '%m/%d/%Y']:
                                try:
                                    date_obj = datetime.strptime(completion_date, fmt)
                                    # Use different format based on whether time is available
                                    if '%H:%M' in fmt or '%H:%M:%S' in fmt:
                                        formatted_date = date_obj.strftime('%d/%m/%Y %H:%M')
                                    else:
                                        formatted_date = date_obj.strftime('%d/%m/%Y')
                                    break
                                except ValueError:
                                    continue
                    elif hasattr(completion_date, 'strftime'):
                        # If it's already a datetime object
                        formatted_date = completion_date.strftime('%d/%m/%Y %H:%M')

                    # If parsing failed, use the original value
                    if not formatted_date:
                        formatted_date = str(completion_date)
                except:
                    formatted_date = str(completion_date)

            # Status styling and icons
            if status == 'completed':
                status_badge = f"<span style='background: #4caf50; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>✓ COMPLETED</span>"
                text_color = "#2e7d32"
                border_color = "#4caf50"
            elif status == 'pending':
                status_badge = f"<span style='background: #ff9800; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>⏳ PENDING</span>"
                text_color = "#333"
                border_color = "#ff9800"
            elif status == 'in_progress':
                status_badge = f"<span style='background: #2196f3; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>🔄 IN PROGRESS</span>"
                text_color = "#1976d2"
                border_color = "#2196f3"
            else:
                status_badge = f"<span style='background: #757575; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>❓ {status.upper()}</span>"
                text_color = "#333"
                border_color = "#757575"

            # Build task item HTML
            task_html = f"""
            <li style='margin-bottom: 12px; padding: 10px; border-left: 3px solid {border_color}; background: #f9f9f9; border-radius: 4px;'>
                <div style='color: {text_color}; font-weight: 500; margin-bottom: 4px;'>
                    <span style='font-size: 0.9em; color: #666;'>Task #{task_id}:</span> {description}
                </div>
                <div style='display: flex; align-items: center; gap: 10px; margin-top: 6px;'>
                    {status_badge}
                    <span style='background: #e3f2fd; color: #1976d2; padding: 1px 6px; border-radius: 8px; font-size: 0.8em; font-weight: bold;'>{points} pts</span>
            """

            # Add completion details if task is completed
            if status == 'completed' and completed_by:
                completion_info = f"<span style='color: #4caf50; font-size: 0.85em; font-weight: 500;'>👤 {completed_by}"
                if formatted_date:
                    completion_info += f" • 📅 {formatted_date}"
                completion_info += "</span>"
                task_html += completion_info

            task_html += "</div></li>"
            formatted_tasks.append(task_html)

        elif isinstance(task, str):
            # If it's a simple string, format it as pending
            formatted_tasks.append(f"""
            <li style='margin-bottom: 12px; padding: 10px; border-left: 3px solid #ff9800; background: #f9f9f9; border-radius: 4px;'>
                <div style='color: #333; font-weight: 500; margin-bottom: 4px;'>{task}</div>
                <div style='display: flex; align-items: center; gap: 10px; margin-top: 6px;'>
                    <span style='background: #ff9800; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>⏳ PENDING</span>
                </div>
            </li>
            """)

    return "".join(formatted_tasks)

# Create the tasks list widget with enhanced styling
tasks_list = create_html_widget(
    f"<ul style='list-style-type: none; padding-left: 0; line-height: 1.6;'>{format_optimization_task_list(daily_tasks)}</ul>"
)

leaderboard_title = create_html_widget("<h3>Engineer Optimization Race Leaderboard</h3>")
leaderboard_items = [f"<li><span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>"
                     for engineer, performance in engineers_performance.items()]
leaderboard = create_html_widget("<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>")

manager_back_btn = create_button("Back to Main Menu", button_style='info')
manager_output = create_output()
manager_container = create_vbox(
    [
        manager_title,
        production_status_title,
        widgets.VBox([sensor_status_display], layout=widgets.Layout(border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', padding='15px', margin_bottom='15px',
                                                                     background_color=WHITE_COLOR)),
        tasks_title,
        tasks_list,
        leaderboard_title,
        leaderboard,
        manager_back_btn,
        manager_output
    ]
)

def update_sensor_status() -> None:
    """Updates the sensor status display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    indoor_status = "Available" if latest_data.get("Indoor Sensor") else "Not Available"
    outdoor_status = "Available" if latest_data.get("Outdoor Sensor") else "Not Available"
    status_text = f"""
        <b>Indoor Sensor:</b> <span style='font-weight:bold;'>{indoor_status}</span><br>
        <b>Outdoor Sensor:</b> <span style='font-weight:bold;'>{outdoor_status}</span>
    """
    sensor_status_display.value = status_text

def update_task_list():
    """Updates the task list display with enhanced formatting"""
    tasks_list.value = f"<ul style='list-style-type: none; padding-left: 0; line-height: 1.6;'>{format_optimization_task_list(daily_tasks)}</ul>"

# --- Anomaly Detection Helper (Reimplemented) ---
def detect_sensor_anomalies(sensor_data: dict) -> list:
    """Returns a list of anomaly messages if any sensor value is out of normal range."""
    anomalies = []
    # Define normal ranges for each attribute
    normal_ranges = {
        "Temperature": (16, 24),   # Celsius
        "Humidity": (30, 50),     # Percent
        "Pressure": (0.9, 1.1),   # Bar
        "Distance": (0, 100),     # Cm
        "DLight": (600,800),    # Lux
    }
    for sensor_name, data in sensor_data.items():
        if not data:
            continue
        for key, (min_v, max_v) in normal_ranges.items():
            value = data.get(key)
            if value is not None:
                try:
                    val = float(value)
                    if val < min_v or val > max_v:
                        anomalies.append(
                            f"<span style='font-weight:bold; color:#fff; background:linear-gradient(90deg,#ff1744,#ff9100); padding:2px 8px; border-radius:6px;'>{sensor_name}:</span> "
                            f"<span style='color:#ff1744; font-weight:bold;'>{key} = {val}</span> "
                            f"<span style='color:#fff; background:#333; border-radius:4px; padding:1px 6px; font-size:0.95em;'>(Normal: {min_v}-{max_v})</span>"
                        )
                except Exception:
                    continue
    return anomalies

# --- Manager Alert Widget (Simple Style, Original) ---
manager_alert = create_html_widget(
    "",
    border_radius='8px',
    padding='10px',
    margin_bottom='10px',
    background_color='#ffeaea',
    box_shadow=BOX_SHADOW
)
def update_manager_leaderboard():
    # Sort by Points (descending), then optionally by Improvements Completed as tiebreaker
    sorted_engineers = sorted(
        engineers_performance.items(),
        key=lambda item: (-item[1]['Points'], -item[1].get('Improvements Completed', 0))
    )

    leaderboard_items = [
        f"<li><span style='font-weight:bold;'>{engineer}</span>: "
        f"<span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, "
        f"<span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>"
        for engineer, performance in sorted_engineers
    ]

    leaderboard.value = "<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>"



def update_manager_alert() -> None:
    """Updates the anomaly alert display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    anomalies = detect_sensor_anomalies(latest_data)
    if anomalies:
        alert_html = (
            "<div style='color:#b71c1c; font-weight:bold; font-size:1.1em;'>"
            "<span style='font-size:1.5em; vertical-align:middle; margin-right:10px;'>⚡</span>"
            "Sensor Anomaly Detected!<br>"
            + "<br>".join(anomalies) +
            "</div>"
        )
        manager_alert.value = alert_html
    else:
        manager_alert.value = ""  # No alert

# --- Add alert widget to manager_container (at the top) ---
manager_container.children = (manager_alert,) + manager_container.children

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    # Add chat_container to the displayed screen children
    displayed_screen.children = [manager_container, chat_container]
    display(displayed_screen)
    update_sensor_status()
    update_task_list()  # Update task list with clean formatting
    update_manager_alert()

In [154]:
# --- Search Query Screen Widgets ---
query_title = create_html_widget("<h2 style='color:#3498db;'>Search Query</h2>")
query_input = widgets.Text(
    description="Enter your query:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(
        width='600px',
        height='40px',
        border_radius=BORDER_RADIUS,
        padding='10px',
        font_size='16px'
    )
)
search_button = create_button("Search")
query_results = create_output(layout=widgets.Layout(height='300px', border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', overflow='auto', background_color='rgba(255,255,255,0.8)', padding='15px'))
query_back_btn = create_button("Back to Main Menu", button_style='info')
query_container = create_vbox(
    [
        query_title,
        query_input,
        search_button,
        query_results,
        query_back_btn
    ]
)

# --- Statistics Screen Widgets ---
stats_title = create_html_widget("<h2 style='color:#2c3e50; text-align:center;'>Sensor Statistics</h2>")
stats_display = create_html_widget(
    "",
    border_radius='8px',
    padding='15px',
    margin_bottom='15px',
    background_color='rgba(255,255,255,0.9)',
    box_shadow=BOX_SHADOW
)


sensor_selector = create_dropdown(
    options=["Indoor Sensor", "Outdoor Sensor"],
    description="<b style='color:#2c3e50;'>Sel_Sensor:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'}, # Added this line
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)

graph_selector = create_dropdown(
    options=["Humidity", "Temperature", "Pressure", "Distance"], # Default options, will be updated
    description="<b style='color:#2c3e50;'>Sel_Graph:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'},
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)



selected_info_display = create_html_widget(
    "",
    border_radius='8px',
    padding='10px',
    margin_bottom='10px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW
)

stats_back_btn = create_button("Back to Main Menu", button_style='info', layout=widgets.Layout(width='200px', height='50px', border_radius='10px', font_size='16px', margin='20px auto'))

# Create a dedicated output widget for plots with proper layout
stats_plots = widgets.Output(layout=widgets.Layout(
    min_height='600px',
    width='100%',
    border=f'1px solid {LIGHT_GREY_BORDER}',
    border_radius='8px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW,
    padding='20px',
    overflow='visible'
))

stats_container = create_vbox(
    [
        stats_title,
        sensor_selector,
        graph_selector,
        selected_info_display,
        stats_display,
        stats_plots,
        stats_back_btn
    ],
    layout=widgets.Layout(padding='30px', border=f'2px solid {DARKER_BLUE_COLOR}', border_radius='15px', margin='20px auto', background_color=LIGHTER_BLUE_COLOR, box_shadow=BOX_SHADOW)
)

In [155]:
# --- Enhanced Optimization Race Screen Widgets ---
optimization_title = create_html_widget("<h2 style='color:#3498db;'>Optimization Race</h2>")
optimization_leaderboard_title = create_html_widget("<h3>Current Standings</h3>")
optimization_leaderboard = create_html_widget("")
optimization_tasks_title = create_html_widget("<h3>Your Daily Tasks</h3>")
optimization_tasks_list = create_html_widget("")
optimization_back_btn = create_button("Back to Main Menu", button_style='info')

# New widgets for task completion
task_completion_title = create_html_widget("<h4>Complete a Task</h4>")
task_dropdown = create_dropdown(
    options=[("-- Select a Task --", None)],
    description="Select Task:",
    value=None,
    style={'description_width': 'auto'}
)

engineer_dropdown = create_dropdown(
    options=[("-- Select Engineer --", None)] + [(name, name) for name in engineers_performance.keys()],
    description="Done by:",
    value=None,
    style={'description_width': 'auto'}
)
complete_task_btn = create_button("Mark as Completed", button_style='success')
optimization_output = create_output()

optimization_container = create_vbox(
    [
        optimization_title,
        optimization_leaderboard_title,
        optimization_leaderboard,
        optimization_tasks_title,
        optimization_tasks_list,
        task_completion_title,
        task_dropdown,
        engineer_dropdown,
        complete_task_btn,
        optimization_back_btn,
        optimization_output
    ]
)

def format_optimization_task_list(tasks):
    """Format tasks using the same enhanced styling as the manager screen"""
    from datetime import datetime

    formatted_tasks = []
    for task in tasks:
        if isinstance(task, dict):
            # Extract task details
            task_id = task.get('id', 'N/A')
            description = task.get('description', 'No description')
            status = task.get('status', 'unknown')
            completed_by = task.get('completed_by', None)
            completion_date = task.get('completion_date', None)
            points = task.get('points', 0)

            # Format completion date to DD/MM/YYYY if it exists
            formatted_date = None
            if completion_date:
                try:
                    # Try to parse various date formats and convert to DD/MM/YYYY
                    if isinstance(completion_date, str):
                        # Try common formats
                        for fmt in ['%Y-%m-%d %H:%M', '%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y-%m-%d %H:%M:%S']:
                            try:
                                date_obj = datetime.strptime(completion_date, fmt)
                                formatted_date = date_obj.strftime('%d/%m/%Y')
                                break
                            except ValueError:
                                continue
                    elif hasattr(completion_date, 'strftime'):
                        # If it's already a datetime object
                        formatted_date = completion_date.strftime('%d/%m/%Y')

                    # If parsing failed, use the original value
                    if not formatted_date:
                        formatted_date = str(completion_date)
                except:
                    formatted_date = str(completion_date)

            # Status styling and icons
            if status == 'completed':
                status_badge = f"<span style='background: #4caf50; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>✓ COMPLETED</span>"
                text_color = "#2e7d32"
                border_color = "#4caf50"
            elif status == 'pending':
                status_badge = f"<span style='background: #ff9800; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>⏳ PENDING</span>"
                text_color = "#333"
                border_color = "#ff9800"
            elif status == 'in_progress':
                status_badge = f"<span style='background: #2196f3; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>🔄 IN PROGRESS</span>"
                text_color = "#1976d2"
                border_color = "#2196f3"
            else:
                status_badge = f"<span style='background: #757575; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>❓ {status.upper()}</span>"
                text_color = "#333"
                border_color = "#757575"

            # Build task item HTML
            task_html = f"""
            <li style='margin-bottom: 12px; padding: 10px; border-left: 3px solid {border_color}; background: #f9f9f9; border-radius: 4px;'>
                <div style='color: {text_color}; font-weight: 500; margin-bottom: 4px;'>
                    <span style='font-size: 0.9em; color: #666;'>Task #{task_id}:</span> {description}
                </div>
                <div style='display: flex; align-items: center; gap: 10px; margin-top: 6px;'>
                    {status_badge}
                    <span style='background: #e3f2fd; color: #1976d2; padding: 1px 6px; border-radius: 8px; font-size: 0.8em; font-weight: bold;'>{points} pts</span>
            """

            # Add completion details if task is completed
            if status == 'completed' and completed_by:
                completion_info = f"<span style='color: #4caf50; font-size: 0.85em; font-weight: 500;'>👤 {completed_by}"
                if formatted_date:
                    completion_info += f" • 📅 {formatted_date}"
                completion_info += "</span>"
                task_html += completion_info

            task_html += "</div></li>"
            formatted_tasks.append(task_html)

        elif isinstance(task, str):
            # If it's a simple string, format it as pending
            formatted_tasks.append(f"""
            <li style='margin-bottom: 12px; padding: 10px; border-left: 3px solid #ff9800; background: #f9f9f9; border-radius: 4px;'>
                <div style='color: #333; font-weight: 500; margin-bottom: 4px;'>{task}</div>
                <div style='display: flex; align-items: center; gap: 10px; margin-top: 6px;'>
                    <span style='background: #ff9800; color: white; padding: 2px 8px; border-radius: 12px; font-size: 0.85em; font-weight: bold;'>⏳ PENDING</span>
                </div>
            </li>
            """)

    return "".join(formatted_tasks)

def update_optimization_tasks_display() -> None:
    """Updates the tasks list display with enhanced manager-style formatting."""
    optimization_tasks_list.value = f"<ul style='list-style-type: none; padding-left: 0; line-height: 1.6;'>{format_optimization_task_list(daily_tasks)}</ul>"

def update_task_dropdown() -> None:
    """Updates the dropdown with pending tasks only and resets selection."""
    pending_tasks = [task for task in daily_tasks if task["status"] == "pending"]
    task_options = [("-- Select a Task --", None)]  # Always include the default option

    if pending_tasks:
        # Add pending tasks to the dropdown
        task_options.extend([(f"{task['description']} ({task['points']} pts)", task['id']) for task in pending_tasks])
    else:
        # Add "No pending tasks" option while keeping the default
        task_options.append(("-- No pending tasks --", None))

    # Clear and rebuild options to ensure proper refresh
    task_dropdown.options = []
    task_dropdown.options = task_options
    task_dropdown.value = None

    # Force refresh the dropdown display
    if hasattr(task_dropdown, 'refresh'):
        task_dropdown.refresh()
    elif hasattr(task_dropdown, 'update'):
        task_dropdown.update()
def complete_task(task_id: int, engineer_name: str) -> bool:
    """Marks a task as completed and awards points to the engineer."""
    from datetime import datetime

    # Find the task
    task_to_complete = None
    for task in daily_tasks:
        if task["id"] == task_id and task["status"] == "pending":
            task_to_complete = task
            break

    if not task_to_complete:
        return False

    # Mark task as completed
    task_to_complete["status"] = "completed"
    task_to_complete["completed_by"] = engineer_name
    # Format completion date as DD/MM/YYYY HH:MM using local timezone
    task_to_complete["completion_date"] = datetime.now(local_tz).strftime("%d/%m/%Y %H:%M")

    # Award points to engineer
    if engineer_name in engineers_performance:
        engineers_performance[engineer_name]["Points"] += task_to_complete["points"]
        engineers_performance[engineer_name]["Improvements Completed"] += 1

    return True
def on_complete_task_clicked(b):
    """Handles the complete task button click."""
    with optimization_output:
        optimization_output.clear_output()

        if not task_dropdown.value:
            print("❌ No task selected or no pending tasks available.")
            return

        if not engineer_dropdown.value:
            print("❌ Please select an engineer.")
            return

        task_id = task_dropdown.value
        engineer_name = engineer_dropdown.value

        if complete_task(task_id, engineer_name):
            print(f"✅ Task completed successfully!")
            print(f"🎉 {engineer_name} earned points!")

            # Update all displays
            update_optimization_tasks_display()
            update_task_dropdown()
            update_optimization_leaderboard()
            update_engineer_dropdown()
            # Clear selections
            task_dropdown.value = None
            engineer_dropdown.value = None
        else:
            print("❌ Failed to complete task. Please try again.")
def update_engineer_dropdown() -> None:
    """Updates the engineer dropdown and resets selection."""
    engineer_options = [("-- Select Engineer --", None)]  # Always include the default option
    engineer_options.extend([(name, name) for name in engineers_performance.keys()])

    # Clear and rebuild options to ensure proper refresh
    engineer_dropdown.options = []
    engineer_dropdown.options = engineer_options
    engineer_dropdown.value = None
def update_optimization_leaderboard() -> None:
    """Updates the leaderboard displayed on the optimization screen."""
    sorted_engineers = sorted(engineers_performance.items(), key=lambda item: item[1]['Points'], reverse=True)
    leaderboard_items = []
    medal_icons = {
        0: "🥇",
        1: "🥈",
        2: "🥉"
    }
    for i, (engineer, performance) in enumerate(sorted_engineers):
        medal = medal_icons.get(i, "")
        leaderboard_items.append(f"<li>{medal} <span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</li>")
    optimization_leaderboard.value = "<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>"

def initialize_optimization_screen() -> None:
    """Initialize the optimization screen with current data."""
    update_optimization_tasks_display()
    update_task_dropdown()
    update_engineer_dropdown()
    update_optimization_leaderboard()

# Connect the button event
complete_task_btn.on_click(on_complete_task_clicked)

# Initialize the screen when first loaded
initialize_optimization_screen()

In [156]:
# --- ScreenManagement Functions ---
current_screen = "main_menu"
displayed_screen = widgets.VBox([])

def show_main_menu() -> None:
    """Displays the main menu screen and updates the current time and dashboard summary."""
    global current_screen, displayed_screen, dashboard_summary # Ensure dashboard_summary is accessible
    current_screen = "main_menu"
    clear_output(wait=True)

    # Update the welcome message with the current time right before displaying
    current_time_str = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
    welcome_msg.value = f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time_str}</p></div>"

    # Re-fetch and update sensor status for the dashboard summary when returning to main menu
    latest_sensor_info = fetch_latest_sensor_data()
    indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
    outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

    # --- Calculate pending_tasks_count HERE before using it in the f-string ---
    # Ensure daily_tasks is globally accessible or passed into this function if needed
    pending_tasks_count = sum(1 for task in daily_tasks if task.get('status') == 'pending')
    # --- End of Calculation ---
    # --- Determine the current Optimization Race leader ---
    leader_name, leader_data = max(engineers_performance.items(), key=lambda item: item[1]["Points"])
    leader_points = leader_data["Points"]

    # Re-create dashboard_summary's value to reflect the new sensor status and task count
    dashboard_summary.value = f"""
        <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
            <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
            <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                    </h4>
                    <p style='margin-bottom:5px; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                    </p>
                    <p style='margin-bottom:0; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                    </p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                    </h4>
                    <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{pending_tasks_count}</span> tasks pending</p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                    </h4>
                   <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>{leader_name}</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>{leader_points}</span> points</p>
                </div>
            </div>
        </div>
        """

    # Add the chatbot button to the main menu container
    main_menu_container.children = [main_menu_title, welcome_msg, dashboard_summary, menu_buttons, chatbot_button]
    displayed_screen.children = [main_menu_container, chat_container]
    display(displayed_screen)

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    # Add chat_container and chatbot_button to the displayed screen children
    manager_container.children = manager_container.children + (chatbot_button,)
    displayed_screen.children = [manager_container, chat_container]
    display(displayed_screen)
    update_sensor_status()
    update_manager_alert()
    update_task_list()
    update_manager_leaderboard()

def show_query_screen() -> None:
    """Displays the query screen."""
    global current_screen, displayed_screen
    current_screen = "query"
    clear_output(wait=True)
    # Add chat_container and chatbot_button to the displayed screen children
    query_container.children = query_container.children + (chatbot_button,)
    displayed_screen.children = [query_container, chat_container]
    display(displayed_screen)

    query_input.value = ""  # Clear the text input box
    query_results.clear_output()  # Clear previous search results

def show_statistics_screen() -> None:
    """Displays the statistics screen."""
    global current_screen, displayed_screen
    current_screen = "statistics"
    clear_output(wait=True)
    # Add chat_container and chatbot_button to the displayed screen children
    stats_container.children = stats_container.children + (chatbot_button,)
    displayed_screen.children = [stats_container, chat_container]
    display(displayed_screen)

    # Initialize graph selector options based on the default sensor
    if sensor_selector.value == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif sensor_selector.value == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLight"]

    # Trigger an initial update for both statistics and plot
    update_sensor_data_and_plot(None)

def show_optimization_screen() -> None:
    """Displays the optimization screen."""
    global current_screen, displayed_screen
    current_screen = "optimization"
    clear_output(wait=True)
    # Add chat_container and chatbot_button to the displayed screen children
    optimization_container.children = optimization_container.children + (chatbot_button,)
    displayed_screen.children = [optimization_container, chat_container]
    display(displayed_screen)
    update_optimization_leaderboard()

def update_screen() -> None:
    """Updates the displayed screen based on the current screen variable."""
    if current_screen == "main_menu":
        show_main_menu()
    elif current_screen == "manager":
        show_manager_screen()
    elif current_screen == "query":
        show_query_screen()
    elif current_screen == "statistics":
        show_statistics_screen()
    elif current_screen == "optimization":
        show_optimization_screen()

In [157]:
# --- Event Handlers ---
def handle_main_menu_navigation(button: widgets.Button) -> None:
    """Handles navigation from the main menu screen."""
    if button.description == "System Manager":
        show_manager_screen()
    elif button.description == "Sensor Statistics":
        show_statistics_screen()
    elif button.description == "Search Query":
        show_query_screen()
        query_input.value = ""  # Clear the text input box
        query_results.clear_output()  # Clear previous search results

    elif button.description == "Optimization Race":
        show_optimization_screen()

def handle_back_to_menu(button: widgets.Button) -> None:
    """Handles back to main menu navigation from any screen."""
    show_main_menu()

def handle_search(button: widgets.Button) -> None:
    """Handles the search query and displays results."""
    query = query_input.value
    results = perform_search(query)
    query_results.clear_output(wait=True)
    with query_results:
        display(widgets.HTML("<br>".join(results)))

def on_sensor_selector_change(change) -> None:
    """Updates graph options and then the plot based on the selected sensor."""
    selected_sensor = change['new']
    if selected_sensor == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif selected_sensor == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLight"]
    # Ensure the graph selector has a valid option selected after options change
    # IMPORTANT: Keep this check to prevent errors if the new sensor doesn't have the previously selected attribute
    if graph_selector.value not in graph_selector.options:
        graph_selector.value = graph_selector.options[0] if graph_selector.options else None

    # Trigger an update for both statistics and plot
    update_sensor_data_and_plot(None) # Call with None as change object is not relevant here

In [158]:
# --- Widget Event Assignments ---
manager_btn.on_click(handle_main_menu_navigation)
statistics_btn.on_click(handle_main_menu_navigation)
search_btn.on_click(handle_main_menu_navigation)
optimization_btn.on_click(handle_main_menu_navigation)

manager_back_btn.on_click(handle_back_to_menu)
stats_back_btn.on_click(handle_back_to_menu)
query_back_btn.on_click(handle_back_to_menu)
optimization_back_btn.on_click(handle_back_to_menu)

search_button.on_click(handle_search)

sensor_selector.observe(on_sensor_selector_change, names='value')
graph_selector.observe(update_sensor_data_and_plot, names='value')

In [159]:
!pip install google-generativeai python-dotenv

import google.generativeai as genai
import os
from IPython.display import display, HTML
import ipywidgets as widgets

# Singleton pattern for the chatbot
class GeminiChatbot:
    _instance = None

    @classmethod
    def get_instance(cls, api_key):
        if cls._instance is None:
            try:
                genai.configure(api_key=api_key)
                cls._instance = genai.GenerativeModel('gemini-pro')
                print("Gemini Chatbot initialized.")
            except Exception as e:
                print(f"Error initializing Gemini Chatbot: {e}")
                cls._instance = None # Ensure instance is None on failure
        return cls._instance

    def send_message(self, message):
        if self._instance:
            try:
                response = self._instance.generate_content(message)
                return response.text
            except Exception as e:
                return f"Error generating response: {e}"
        else:
            return "Chatbot is not initialized. Please check your API key."

# Get API key from Google Secrets
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except Exception as e:
    print(f"Could not retrieve GEMINI_API_KEY from Google Secrets: {e}")
    print("Please ensure the secret 'GEMINI_API_KEY' is set in Colab.")
    GEMINI_API_KEY = None

# Initialize the chatbot (will only create instance if API key is available)
chatbot = GeminiChatbot.get_instance(GEMINI_API_KEY)

# Create Chatbot UI
chat_output = widgets.Output(layout=widgets.Layout(height='300px', overflow_y='auto', border='1px solid #ccc', padding='10px'))
chat_input = widgets.Text(
    placeholder='Ask me a question about the dashboard...',
    layout=widgets.Layout(width='calc(100% - 50px)')
)
send_button = widgets.Button(icon='send', layout=widgets.Layout(width='40px'))
chat_area = widgets.VBox([chat_output, widgets.HBox([chat_input, send_button])])

# Chatbot container with title bar and close button
chat_container = widgets.VBox([
    widgets.HBox([
        widgets.HTML("<b>Gemini Chatbot</b>", layout=widgets.Layout(flex='1')),
        widgets.Button(icon='times', layout=widgets.Layout(width='30px'), button_style='danger')
    ], layout=widgets.Layout(justify_content='space-between', align_items='center', padding='0 10px', background_color='#3498db', color='white', border_top_left_radius='8px', border_top_right_radius='8px')),
    chat_area
], layout=widgets.Layout(
    display='none', # Initially hidden
    position='fixed',
    bottom='80px',  # Adjust position to be above the chatbot button
    right='20px',
    width='400px',
    border='1px solid #ccc',
    box_shadow='0px 4px 8px rgba(0,0,0,0.3)',
    border_radius='8px',
    background_color='white',
    z_index=1000 # Ensure it's on top
))

# Chatbot open/close button
chatbot_button = widgets.Button(
    icon='comment-dots',
    button_style='info',
    tooltip='Open Chatbot',
    layout=widgets.Layout(
        # Positioning for bottom-right corner
        position='fixed',
        bottom='20px',
        right='20px',
        width='60px',
        height='60px',
        border_radius='30px',
        box_shadow='0px 4px 8px rgba(0,0,0,0.3)',
        z_index=999 # Below chatbot window
    )
)

def toggle_chatbot_visibility(b):
    if chat_container.layout.display == 'none':
        chat_container.layout.display = 'flex'
        chatbot_button.tooltip = 'Close Chatbot'
        chatbot_button.icon = 'times'
        chatbot_button.button_style = 'danger'
        # Add initial message to the chat window
        with chat_output:
            chat_output.clear_output()
            display(HTML("<div style='margin-bottom: 10px; padding: 8px; background-color: #e9e9eb; border-radius: 8px; max-width: 80%;'>Hi there! Ask me anything about the Robotics Lab Dashboard.</div>"))

    else:
        chat_container.layout.display = 'none'
        chatbot_button.tooltip = 'Open Chatbot'
        chatbot_button.icon = 'comment-dots'
        chatbot_button.button_style = 'info'

# Connect toggle button
chatbot_button.on_click(toggle_chatbot_visibility)
chat_container.children[0].children[1].on_click(toggle_chatbot_visibility) # Connect close button in title bar

def handle_send_message(b):
    user_message = chat_input.value.strip()
    chat_input.value = "" # Clear input field

    if user_message:
        with chat_output:
            # Display user message
            display(HTML(f"<div style='text-align: right; margin-bottom: 10px; padding: 8px; background-color: #007bff; color: white; border-radius: 8px; max-width: 80%; margin-left: auto;'>{user_message}</div>"))

            # Get and display chatbot response
            response = chatbot.send_message(user_message)
            display(HTML(f"<div style='margin-bottom: 10px; padding: 8px; background-color: #e9e9eb; border-radius: 8px; max-width: 80%;'>{response}</div>"))

# Connect send button and input field (on enter press)
send_button.on_click(handle_send_message)
chat_input.observe(lambda change: handle_send_message(None) if change['new'] and change['name'] == 'value' and change['value'].endswith('\n') else None, names='value')

# Add the chatbot button and container to the displayed screen (this will make them visible)
# displayed_screen.children = displayed_screen.children + (chatbot_button, chat_container)

Could not retrieve GEMINI_API_KEY from Google Secrets: Secret GEMINI_API_KEY does not exist.
Please ensure the secret 'GEMINI_API_KEY' is set in Colab.
Gemini Chatbot initialized.


<h2>Section 12: Initial Display and Startup</h2>
<p>This final section triggers the initial display of the main menu when the notebook is run. It ensures that users are greeted with the dashboard interface immediately, providing a seamless start to their interaction with the system.</p>

In [160]:
# --- Initial Display ---
show_main_menu()

In [161]:
# !pip install paho-mqtt
# !pip install firebase
# import time
# import json
# import paho.mqtt.client as mqtt
# from datetime import datetime
# import pytz
# from firebase import firebase


# class MqttConnection:
#     _instance = None  # Singleton instance

#     @classmethod
#     def get_instance(cls, DBLink=None):
#         if cls._instance is None and DBLink is not None:
#             cls._instance = cls(DBLink)
#         return cls._instance

#     def __init__(self, DBLink):
#         if MqttConnection._instance is not None:
#             raise Exception("MqttConnection is a singleton.")

#         self.DBLink = DBLink
#         self.FBconn = firebase.FirebaseApplication(DBLink, None)
#         self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
#         self.connected = False

#     def insert_to_db(self, path, data):
#         try:
#             # Read existing data at this path
#             existing = self.FBconn.get('/', f'Data/{path}')
#             if existing is None:
#                 existing = {}

#             # Merge existing data with new data (new data overwrites existing keys)
#             merged_data = {**existing, **data}

#             self.FBconn.put('/', f'Data/{path}', merged_data)
#             print(f"Data saved to Firebase at path: Data/{path}")
#         except Exception as e:
#             print(f"Failed to insert data to Firebase: {e}")



#     def on_connect(self, client, userdata, flags, rc, properties=None):
#         if rc == 0:
#             self.connected = True
#             print("✅ Connected to MQTT Broker. Subscribing to topics...")
#             client.subscribe("braude/D106/indoor")
#             client.subscribe("braude/D106/outdoor")
#             print("✅ Successfully subscribed to topics.")
#         else:
#             print(f"❌ Failed to connect, return code {rc}")
#             self.connected = False

#     def on_disconnect(self, client, userdata, rc, properties=None, reason_string=None):
#         self.connected = False
#         if rc != 0:
#             for i in range(5):
#                 wait_time = (i + 1) * 2
#                 print(f"🔄 Disconnected. Reconnecting in {wait_time}s (attempt {i+1}/5)...")
#                 time.sleep(wait_time)
#                 try:
#                     client.reconnect()
#                     print("✅ Reconnection attempt made.")
#                     break
#                 except Exception as e:
#                     print(f"❌ Reconnection failed: {e}")

#     # Moved inside the class
#     def on_message(self, client, userdata, msg):
#         topic = msg.topic
#         payload = msg.payload.decode('utf-8')
#         try:
#             sensor_data = json.loads(payload)
#             timestamp = int(time.time())

#             local_tz = pytz.timezone("Israel")
#             sensor_data["Time"] = datetime.now(local_tz).strftime("%H:%M:%S")
#             sensor_data["Date"] = datetime.now(local_tz).strftime("%Y-%m-%d")
#             print(sensor_data)
#             if topic == "braude/D106/indoor":
#                 # Now self refers to the MqttConnection instance
#                 self.insert_to_db(f"indoor/{timestamp}", sensor_data)
#             elif topic == "braude/D106/outdoor":
#                 # Now self refers to the MqttConnection instance
#                 self.insert_to_db(f"outdoor/{timestamp}", sensor_data)
#         except json.JSONDecodeError as e:
#             print(f"❌ JSON decode error: {e}\nPayload: {payload}")

#     # Moved inside the class
#     def mqtt_handler(self):
#         self.client.on_connect = self.on_connect
#         self.client.on_disconnect = self.on_disconnect
#         self.client.on_message = self.on_message

#         try:
#             print("🔌 Attempting to connect to MQTT broker...")
#             conn_result = self.client.connect("test.mosquitto.org", 1883, keepalive=60)
#             print(f"Connection result code: {conn_result}")
#             self.client.loop_start()

#             timeout = time.time() + 10
#             while time.time() < timeout and not self.connected:
#                 time.sleep(0.2)

#             return self.connected

#         except Exception as e:
#             print(f"❌ Connection failed: {e}")
#             return False


# # --- Main execution ---
# if __name__ == "__main__":
#     DBLink = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"

#     print("🚀 Initializing MQTT connection...")
#     global_mqtt_connection = MqttConnection.get_instance(DBLink)

#     if not global_mqtt_connection.connected:
#         success = global_mqtt_connection.mqtt_handler()
#         print(f"🔍 MQTT connection result: {'Connected' if success else 'Failed'}")

#     # Let it run for a while to receive messages
#     try:
#         time.sleep(30)
#     finally:
#         global_mqtt_connection.client.loop_stop()
#         print("🛑 MQTT loop stopped.")

In [162]:
# import time
# !pip install firebase
# from firebase import firebase
# from datetime import datetime
# from zoneinfo import ZoneInfo  # Python 3.9+
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import random

# # Connect to Firebase
# FBconn = firebase.FirebaseApplication(
#     'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
# )

# local_tz = ZoneInfo("Asia/Jerusalem")

# def generate_logical_data_points(n=20):
#     data_points = []

#     base_humidity = 41.16
#     base_temperature = 23.2
#     base_pressure = 1
#     base_distance = 50

#     for i in range(n):
#         # Simulate small random fluctuations
#         humidity = base_humidity + random.uniform(-0.2, 0.2)
#         temperature = base_temperature + random.uniform(-0.05, 0.05)
#         pressure = base_pressure + random.uniform(-0.1, 0.1)
#         distance = base_distance + random.uniform(-50, 50)

#         point = {
#             "Humidity": round(humidity, 2),
#             "Temperature": round(temperature, 2),
#             "Pressure": round(pressure, 2),
#             "Distance": round(distance, 2)
#         }
#         data_points.append(point)

#     return data_points

# def add_data_points(data_points_raw, delay=1):
#     for point in data_points_raw:
#         now = datetime.now(local_tz)
#         timestamp = int(time.time())

#         # Separate date and time strings (no milliseconds)
#         date_str = now.strftime("%Y-%m-%d")
#         time_str = now.strftime("%H:%M:%S")

#         # Clamp temperature between 20 and 25
#         temp = max(23.0, min(25.0, point["Temperature"]))
#         humidity = point["Humidity"]
#         pressure = point["Pressure"]
#         distance = point["Distance"]

#         indoor_data = {
#             "Temperature": round(temp, 2),
#             "Humidity": round(humidity, 2),
#             "Pressure": round(pressure, 2),
#             "Distance": round(distance, 2),
#             "Date": date_str,
#             "Time": time_str
#         }

#         outdoor_temp = max(23.0, min(25.0, temp + random.uniform(-0.5, 0.5)))
#         outdoor_humidity = max(41.0, min(43.0, humidity + random.uniform(-0.2, 0.2)))
#         dlight = random.randint(600, 800)

#         outdoor_data = {
#             "Temperature": round(outdoor_temp, 2),
#             "Humidity": round(outdoor_humidity, 2),
#             "DLight": dlight,
#             "Date": date_str,
#             "Time": time_str
#         }

#         FBconn.put("Data/indoor", timestamp, indoor_data)
#         FBconn.put("Data/outdoor", timestamp, outdoor_data)

#         print(f"Uploaded indoor and outdoor data at timestamp: {timestamp}")

#         time.sleep(5)

# data_points = generate_logical_data_points(50)  # Generate 50 points for testing
# add_data_points(data_points, delay=5)  # 5 second delay between uploads

In [163]:
# !pip install requests beautifulsoup4 nltk firebase

# # Import modules
# import requests
# from bs4 import BeautifulSoup
# import re
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# from firebase import firebase

# # Download stopwords
# nltk.download('stopwords')
# english_stop_words = set(stopwords.words('english'))

# # Firebase setup
# firebase_url = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"
# FBconn = firebase.FirebaseApplication(firebase_url, None)

# # Pages to index
# seed_urls = {
#     1: 'https://mqtt.org/',
#     2: 'https://mqtt.org/documentation',
#     3: 'https://mqtt.org/software',
#     4: 'https://mqtt.org/getting-started/',
#     5: 'https://mqtt.org/mqtt-specification/',
#     6: 'https://mqtt.org/use-cases/',
#     7: 'https://mqtt.org/faq/',
# }

# # Fetch page content
# def fetch_page_text(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
#             return soup.get_text()
#     except Exception as e:
#         print(f"Failed to fetch {url}: {e}")
#     return ""
# !pip install requests beautifulsoup4 nltk firebase
# import requests
# from bs4 import BeautifulSoup
# import re
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# from firebase import firebase
# nltk.download('stopwords')
# english_stop_words = set(stopwords.words('english'))
# firebase_url = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"
# FBconn = firebase.FirebaseApplication(firebase_url, None)
# seed_urls = {
#     1: 'https://mqtt.org/',
#     2: 'https://mqtt.org/documentation',
#     3: 'https://mqtt.org/software',
#     4: 'https://mqtt.org/getting-started/',
#     5: 'https://mqtt.org/mqtt-specification/',
#     6: 'https://mqtt.org/use-cases/',
#     7: 'https://mqtt.org/faq/',
# }
# def fetch_page_text(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
#             return soup.get_text()
#     except Exception as e:
#         print(f"Failed to fetch {url}: {e}")
#     return ""
# def sanitize_key(key: str) -> str:
#     return re.sub(r'[.#$]', '_', key)
# def build_index(urls):
#     index = {}
#     stemmer = PorterStemmer()
#     for _, url in urls.items():
#         print(f"Fetching and processing {url} ...")
#         text = fetch_page_text(url)
#         words = re.findall(r'\w+', text.lower())
#         for word in words:
#             if word in english_stop_words or len(word) <= 2:
#                 continue
#             stemmed = stemmer.stem(word)
#             if stemmed not in index:
#                 index[stemmed] = {
#                     "term": stemmed,
#                     "DocIDs": {},
#                     "count": 0
#                 }
#             index[stemmed]["DocIDs"][url] = index[stemmed]["DocIDs"].get(url, 0) + 1
#             index[stemmed]["count"] += 1
#     for data in index.values():
#         data["DocIDs"] = [
#             {"url": url, "count": count}
#             for url, count in sorted(data["DocIDs"].items())
#         ]
#     return index
# full_index = build_index(seed_urls)
# top_100_terms = sorted(full_index.values(), key=lambda x: x["count"], reverse=True)[:100]
# top_100_index = {
#     sanitize_key(entry["term"]): {
#         "term": entry["term"],
#         "count": entry["count"],
#         "DocIDs": entry["DocIDs"]
#     }
#     for entry in top_100_terms
# }
# print("\nUploading Top 100 words to Firebase...")
# FBconn.put('/dataWarehouse', 'Index', top_100_index)
# print("Upload complete.")
# print("\nTop 100 words uploaded:")
# for entry in top_100_terms:
#     print(f"{entry['term']:15} → {entry['count']:4} total occurrences in:")
#     for site in entry["DocIDs"]:
#         print(f"  {site['url']} → {site['count']} times")
# # Build full index using URLs as "DocIDs"
# def build_index(urls):
#     index = {}
#     stemmer = PorterStemmer()

#     for _, url in urls.items():
#         print(f"Fetching and processing {url} ...")
#         text = fetch_page_text(url)
#         words = re.findall(r'\w+', text.lower())

#         for word in words:
#             if word in english_stop_words or len(word) <= 2:
#                 continue
#             stemmed = stemmer.stem(word)
#             if stemmed not in index:
#                 index[stemmed] = {
#                     "term": stemmed,
#                     "DocIDs": {},
#                     "count": 0
#                 }
#             # Count occurrences per URL
#             index[stemmed]["DocIDs"][url] = index[stemmed]["DocIDs"].get(url, 0) + 1
#             index[stemmed]["count"] += 1

#     # Convert DocIDs dict to sorted list of URLs + counts
#     for data in index.values():
#         data["DocIDs"] = [
#             {"url": url, "count": count}
#             for url, count in sorted(data["DocIDs"].items())
#         ]

#     return index

# # Build and sort index
# full_index = build_index(seed_urls)

# # Get top 100 terms by frequency
# top_100_terms = sorted(full_index.values(), key=lambda x: x["count"], reverse=True)[:100]

# # Prepare upload dict with sanitized keys for Firebase compatibility
# top_100_index = {
#     sanitize_key(entry["term"]): {
#         "term": entry["term"],
#         "count": entry["count"],
#         "DocIDs": entry["DocIDs"]
#     }
#     for entry in top_100_terms
# }

# # Upload to Firebase under dataWarehouse/Index
# print("\nUploading Top 100 words to Firebase...")
# FBconn.put('/Data', 'Index', top_100_index)
# print("Upload complete.")

# # Display results
# print("\nTop 100 words uploaded:")
# for entry in top_100_terms:
#     print(f"{entry['term']:15} → {entry['count']:4} total occurrences in:")
#     for site in entry["DocIDs"]:
#         print(f"  {site['url']} → {site['count']} times")